## Revise train data judgement



In [132]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [133]:
input_dir = "/content/drive/MyDrive/07_Competition/signate-471/data/"
output_dir = "/content/drive/MyDrive/07_Competition/signate-471/log/"
submission_dir = "/content/drive/MyDrive/07_Competition/signate-471/submission/"
model_dir = "/content/drive/MyDrive/07_Competition/signate-471/model_bin/"
pred_dir = "/content/drive/MyDrive/07_Competition/signate-471/pred/"

In [134]:
!pip install transformers pycld2
!pip install nlplot 

In [135]:
import os
import math
import random
import pandas as pd
import numpy as np
from glob import glob
import gc
gc.enable()

import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim as optim
from torch.optim.optimizer import Optimizer
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import Dataset, DataLoader, SequentialSampler, RandomSampler

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import fbeta_score

from IPython.display import clear_output
from tqdm import tqdm, trange
import spacy
from sklearn.metrics.pairwise import cosine_similarity

import re
import nltk
from nltk.corpus import stopwords
from scipy.optimize import minimize, minimize_scalar
import regex
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [145]:
feature_df = pd.read_pickle(input_dir+"frature_word_df.pkl")

In [146]:
feature_df.head()

,-PRON-,aa,aaa,aacr,aad,aada,aagp,aai,aami,aan,aao,aap,aas,aat,aatf,aav,ab,abacavir,abad,abalphac,abandon,abate,abatement,abbott,abbreviate,abbreviation,abbvie,abc,abca,abcb,abcd,abcg,abdoman,abdomen,abdominal,abdominopelvic,abducen,abeat,abegrin,aberrant,...,zero,zeta,zevalin,zhang,zhao,zhejiang,zhongnan,zhou,zhu,zhuhai,zidovudine,zika,zikv,zinc,zingiberis,zip,zipper,zj,zl,zla,zlb,zmk,zn,znf,znpc,znt,zona,zonal,zone,zooepidemicus,zoom,zoonosis,zoonotic,zoster,zr,zung,zwitterionic,zymogen,zymography,zymosan
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [147]:
cols = ['pcr', 'samples', 'detection', 'assay', 'accuracy', 'assays', 'tested',  'detect']

feature_df = feature_df[cols]

In [148]:
feature_df["sum_word"] = feature_df.sum(axis=1)
feature_df

,pcr,samples,detection,assay,accuracy,assays,tested,detect,sum_word
0,0,0,1,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
26916,0,0,0,0,0,0,0,0,0
26917,0,0,0,0,0,0,0,0,0
26918,0,0,0,1,0,0,0,0,1
26919,0,0,0,0,0,0,0,0,0


In [149]:
feature_df.sum_word.unique()

array([1, 0, 4, 2, 3, 5, 6])

In [150]:
feature_df = feature_df[feature_df.sum_word > 2]

In [151]:
feature_df

,pcr,samples,detection,assay,accuracy,assays,tested,detect,sum_word
16,1,0,1,1,0,0,0,1,4
296,1,0,1,0,0,0,0,1,3
469,0,1,0,1,0,0,0,1,3
511,1,0,1,1,0,1,0,1,5
569,1,0,0,1,0,0,0,1,3
...,...,...,...,...,...,...,...,...,...
26680,0,1,1,1,0,0,0,1,4
26829,0,0,1,1,0,1,0,1,4
26848,1,1,1,0,0,0,0,0,3
26897,0,0,1,0,0,1,0,1,3


In [152]:
train = pd.read_pickle(input_dir+"train_pre.pkl")
len(train)

26921

In [153]:
train_1 = train[train.judgement == 1]
positive_ids = list(train_1["id"])

In [154]:
id_list=list(feature_df.index)

In [155]:
result_ids = list(set(id_list) - set(positive_ids))
result_ids

[12291,
 16388,
 24591,
 12326,
 18474,
 24618,
 20530,
 10291,
 26680,
 24634,
 4172,
 18520,
 16480,
 24673,
 4196,
 4197,
 16484,
 18554,
 24712,
 8329,
 6291,
 12464,
 18613,
 22713,
 20670,
 26829,
 4306,
 22739,
 14550,
 22746,
 26848,
 18665,
 16619,
 8430,
 6387,
 20725,
 10500,
 24836,
 24837,
 16652,
 26897,
 6423,
 26904,
 6425,
 12574,
 296,
 22824,
 14636,
 24876,
 24884,
 8502,
 20790,
 14662,
 10567,
 10569,
 8524,
 20814,
 20816,
 14673,
 24924,
 12640,
 24938,
 18811,
 6538,
 10636,
 12690,
 4500,
 4503,
 16796,
 24995,
 4519,
 25004,
 20914,
 10683,
 20924,
 6591,
 14785,
 2502,
 12753,
 469,
 12757,
 14806,
 2521,
 16869,
 23017,
 14826,
 12782,
 8691,
 16885,
 14843,
 511,
 6662,
 16904,
 21016,
 12825,
 12827,
 8741,
 16944,
 4658,
 569,
 8763,
 4672,
 2625,
 23106,
 25158,
 2640,
 4690,
 23122,
 4697,
 14943,
 6754,
 14953,
 10859,
 21100,
 6768,
 14962,
 4726,
 8823,
 10887,
 4745,
 17033,
 25226,
 21134,
 8849,
 8850,
 15010,
 17069,
 10932,
 25268,
 698,
 15038

In [156]:
len(result_ids)

382

In [157]:
def filter_id(x):
    if x in result_ids:
        x=1
        return x
    else:
        x=0
        return x

train["judgement_1"] = train["id"].apply(lambda x: filter_id(x))

In [158]:
train

,id,title,abstract,judgement,title_abst,title_abst_pre,title_word_len,title_lang,judgement_1
0,0,One-year age changes in MRI brain volumes in o...,Longitudinal studies indicate that declines in...,0,One-year age changes in MRI brain volumes in o...,"[year, age, change, mri, brain, volume, old, a...",10,en,0
1,1,Supportive CSF biomarker evidence to enhance t...,The present study was undertaken to validate t...,0,Supportive CSF biomarker evidence to enhance t...,"[supportive, csf, biomarker, evidence, enhance...",23,en,0
2,2,Occurrence of basal ganglia germ cell tumors w...,Objective: To report a case series in which ba...,0,Occurrence of basal ganglia germ cell tumors w...,"[occurrence, basal, ganglia, germ, cell, tumor...",10,en,0
3,3,New developments in diagnosis and therapy of C...,The etiology and pathogenesis of idiopathic ch...,0,New developments in diagnosis and therapy of C...,"[new, development, diagnosis, therapy, crohn, ...",13,en,0
4,4,Prolonged shedding of SARS-CoV-2 in an elderly...,NaN,0,Prolonged shedding of SARS-CoV-2 in an elderly...,"[prolong, shedding, sars, cov, elderly, liver,...",16,en,0
...,...,...,...,...,...,...,...,...,...
27140,27140,The amyloidogenic pathway of amyloid precursor...,Amyloid beta-protein (A beta) is the main cons...,0,The amyloidogenic pathway of amyloid precursor...,"[amyloidogenic, pathway, amyloid, precursor, p...",15,en,0
27141,27141,Technologic developments in radiotherapy and s...,We present a review of current technological p...,0,Technologic developments in radiotherapy and s...,"[technologic, development, radiotherapy, stere...",7,en,0
27142,27142,Novel screening cascade identifies MKK4 as key...,Phosphorylation of Tau at serine 422 promotes ...,0,Novel screening cascade identifies MKK4 as key...,"[novel, screening, cascade, identify, mkk, key...",13,en,0
27143,27143,Visualization of the gall bladder on F-18 FDOP...,The ability to label dihydroxyphenylalanine (D...,0,Visualization of the gall bladder on F-18 FDOP...,"[visualization, gall, bladder, f, fdopa, pet, ...",13,en,0


In [159]:
train["judgement"]=train["judgement"]+train["judgement_1"]
train= train.iloc[:,:-1]
train

,id,title,abstract,judgement,title_abst,title_abst_pre,title_word_len,title_lang
0,0,One-year age changes in MRI brain volumes in o...,Longitudinal studies indicate that declines in...,0,One-year age changes in MRI brain volumes in o...,"[year, age, change, mri, brain, volume, old, a...",10,en
1,1,Supportive CSF biomarker evidence to enhance t...,The present study was undertaken to validate t...,0,Supportive CSF biomarker evidence to enhance t...,"[supportive, csf, biomarker, evidence, enhance...",23,en
2,2,Occurrence of basal ganglia germ cell tumors w...,Objective: To report a case series in which ba...,0,Occurrence of basal ganglia germ cell tumors w...,"[occurrence, basal, ganglia, germ, cell, tumor...",10,en
3,3,New developments in diagnosis and therapy of C...,The etiology and pathogenesis of idiopathic ch...,0,New developments in diagnosis and therapy of C...,"[new, development, diagnosis, therapy, crohn, ...",13,en
4,4,Prolonged shedding of SARS-CoV-2 in an elderly...,NaN,0,Prolonged shedding of SARS-CoV-2 in an elderly...,"[prolong, shedding, sars, cov, elderly, liver,...",16,en
...,...,...,...,...,...,...,...,...
27140,27140,The amyloidogenic pathway of amyloid precursor...,Amyloid beta-protein (A beta) is the main cons...,0,The amyloidogenic pathway of amyloid precursor...,"[amyloidogenic, pathway, amyloid, precursor, p...",15,en
27141,27141,Technologic developments in radiotherapy and s...,We present a review of current technological p...,0,Technologic developments in radiotherapy and s...,"[technologic, development, radiotherapy, stere...",7,en
27142,27142,Novel screening cascade identifies MKK4 as key...,Phosphorylation of Tau at serine 422 promotes ...,0,Novel screening cascade identifies MKK4 as key...,"[novel, screening, cascade, identify, mkk, key...",13,en
27143,27143,Visualization of the gall bladder on F-18 FDOP...,The ability to label dihydroxyphenylalanine (D...,0,Visualization of the gall bladder on F-18 FDOP...,"[visualization, gall, bladder, f, fdopa, pet, ...",13,en


In [160]:
len(train[train.judgement == 1])

1005

In [161]:
train.to_pickle(input_dir+"train_rev_2.pkl")

In [131]:
train.iloc[21384,].abstract

'OBJECTIVES: To determine the antibody response against SARS-CoV-2 spike protein and nucleoprotein using 4 automated immunoassays and 3 ELISAs for the detection of total Ig antibodies (Roche) or IgG (Abbott, Diasorin, Snibe, Euroimmun, Mikrogen) in COVID-19 patients. METHODS: Sensitivity and dynamic trend to seropositivity were evaluated in 233 samples from 114 patients with moderate, severe, or critical COVID-19 confirmed with PCR on nasopharyngeal swab. Specificity was evaluated in 113 samples collected before January 2020 including 24 samples from patients with non-SARS coronavirus infection. RESULTS: Sensitivity for all assays was 100% [95% confidence interval: 83.7-100] 3 weeks after onset of symptoms. Specificity varied between 94.7% [88.7-97.8] and 100% [96.1-100]. Calculated at the cut-offs that corresponded to a specificity of 95% and 97.5%, Roche had the highest sensitivity (85.0% [79.8-89.0] and 81.1% [76.6-85.7], p <0.05 except vs. Abbott). Seroconversion occurred on averag